## A JSON format compatible with the pandas data structure 
-------     
      
### Introduction
The data type is not explicitely taken into account in the current JSON interface.     
    
The existing solution is to use a data schema in addition.

### Proposal
To have a simple, compact and reversible solution, I propose to use the [JSON-NTV format (Named and Typed Value)](https://github.com/loco-philippe/NTV#readme) - which integrates the notion of type - and its JSON-TAB variation for tabular data.    
This solution allows to include a large number of types (not necessarily Pandas dtype).     
JSON-TAB is also applicable for multidimensional data (e.g. Xarray).

### Content
This NoteBook uses examples to present some key points

*(active link on jupyter Notebook or Nbviewer)*
- [0 - Simple example](#0---Simple-example)
- [1 - Current Json interface](#1---Current-Json-interface)
    - [Example : simple column](#Example-:-simple-column)
    - [Types and Json interface](#Types-and-Json-interface)
    - [Data compactness](#data-compactness)
    - [External types](#external-types)
- [2 - Series](#2---Series)
    - [Simple example](#Simple-example)
    - [Typed example](#Typed-example)
    - [Examples with a non-Pandas type](#Examples-with-a-non-Pandas-type)
    - [Categorical examples](#Categorical-examples)
- [3 - DataFrame](#3---DataFrame)
    - [Initial example](#Initial-example)
    - [Complete example](#Complete-example)
    - [Json data can be annotated](#Json-data-can-be-annotated)
    - [Categorical data can be included](#Categorical-data-can-be-included)
    - [Multidimensional data](#Multidimensional-data)
- [4 - Appendix : Series tests](#4---Appendix-:-Series-tests)     
- [5 - Appendix: JSON interface - option: "table"](#5---Appendix:-JSON-interface---option:-"table")
    - [Converting pandas dtype to table-schema type](#Converting-pandas-dtype-to-table-schema-type)
    - [Converting table-schema type to pandas dtype](#Converting-table-schema-type-to-pandas-dtype)
    - [Example converting pandas dtype to table-schema type](#Example-converting-pandas-dtype-to-table-schema-type)
        
### References
- [JSON-NTV specification](https://github.com/loco-philippe/NTV/blob/main/documentation/JSON-NTV-standard.pdf)
- [JSON-TAB specification](https://github.com/loco-philippe/NTV/blob/main/documentation/JSON-TAB-standard.pdf)
- [JSON-NTV classes and methods](https://loco-philippe.github.io/NTV/json_ntv.html)

This Notebook can also be viewed at [nbviewer](http://nbviewer.org/github/loco-philippe/NTV/tree/main/example)

In [1]:
import math
from pprint import pprint
from json_ntv import Ntv, NtvConnector
from json_ntv import as_def_type as as_def_type
from json_ntv import read_json as read_json        
from json_ntv import to_json as to_json
import pandas as pd
from shapely.geometry import Point
from datetime import date, datetime, time

## 0 - Simple example

- The example is a Dataframe with several dtype

In [2]:
tab_data = {'index':           [100, 200, 300, 400, 500, 600],
            'dates::date':     pd.Series([date(1964,1,1), date(1985,2,5), date(2022,1,21), date(1964,1,1), date(1985,2,5), date(2022,1,21)]), 
            'value':           [10, 10, 20, 20, 30, 30],
            'value32':         pd.Series([12, 12, 22, 22, 32, 32], dtype='int32'),
            '::month':         pd.Series([1, 2, 1, 1, 2, 1], dtype='category'),
            'coord::point':    pd.Series([Point(1,2), Point(3,4), Point(5,6), Point(7,8), Point(3,4), Point(5,6)]),
            'names':           pd.Series(['john', 'eric', 'judith', 'mila', 'hector', 'maria'], dtype='string'),
            'unique':          True }
df = pd.DataFrame(tab_data).set_index('index')
print(df.dtypes)
df

dates::date       object
value              int64
value32            int32
::month         category
coord::point      object
names             string
unique              bool
dtype: object


,dates::date,value,value32,::month,coord::point,names,unique
index,,,,,,,
100,1964-01-01,10,12,1,POINT (1 2),john,True
200,1985-02-05,10,12,2,POINT (3 4),eric,True
300,2022-01-21,20,22,1,POINT (5 6),judith,True
400,1964-01-01,20,22,1,POINT (7 8),mila,True
500,1985-02-05,30,32,2,POINT (3 4),hector,True
600,2022-01-21,30,32,1,POINT (5 6),maria,True


- the example has a simple and compact JSON representation including dtype

In [3]:
df_to_json = to_json(df)
pprint(df_to_json, width=120)

{':tab': [{'index': [100, 200, 300, 400, 500, 600]},
          {'dates::date': ['1964-01-01', '1985-02-05', '2022-01-21', '1964-01-01', '1985-02-05', '2022-01-21']},
          {'value': [10, 10, 20, 20, 30, 30]},
          {'value32::int32': [12, 12, 22, 22, 32, 32]},
          [{'::month': [1, 2]}, [0, 1, 0, 0, 1, 0]],
          {'coord::point': [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0], [3.0, 4.0], [5.0, 6.0]]},
          {'names::string': ['john', 'eric', 'judith', 'mila', 'hector', 'maria']},
          {'unique': True}]}


- The json conversion is reversible : df_from_json equals initial df

In [4]:
df_from_json = read_json(df_to_json)
print('df created from JSON-NTV is equal to initial df ? ', df_from_json.equals(df))
df_from_json

df created from JSON-NTV is equal to initial df ?  False


,dates::date,value,value32,::month,coord::point,names,unique
100,1964-01-01,10,12,1,POINT (1 2),john,True
200,1985-02-05,10,12,2,POINT (3 4),eric,True
300,2022-01-21,20,22,1,POINT (5 6),judith,True
400,1964-01-01,20,22,1,POINT (7 8),mila,True
500,1985-02-05,30,32,2,POINT (3 4),hector,True
600,2022-01-21,30,32,1,POINT (5 6),maria,True


## 1 - Current Json interface
- the `orient='table'` option of the json interface is detailed in [Appendix](#5---Appendix:-JSON-interface---option:-"table")

### Example : simple column
- the interface is not reversible with this example

In [5]:
df = pd.read_json('{"test integer":[1,2,3], "test string": ["a", "b", "c"]}')
print(df)
# but it is impossible with to_json() to recreate the initial data

   test integer test string
0             1           a
1             2           b
2             3           c


### Types and Json interface 

- the only way to keep the types in the json interface is to use the orient='table' option

In [6]:
df  = pd.DataFrame(pd.Series([10,20], name='int32', dtype='Int32'))

# dtype is not included in usual json interface
df.to_json()

'{"int32":{"0":10,"1":20}}'

- only few types are allowed in json-table interface : int64, float64, bool, datetime64, timedelta64, categorical

In [7]:
# 'int32' is lost in json-table interface
df2 = pd.read_json(df.to_json(orient='table'), orient='table')
print(df2.dtypes)
print('\nis Json translation reversible ? ', df.equals(df2))

int32    int64
dtype: object

is Json translation reversible ?  False


- allowed types are not always kept in json interface 

In [8]:
df = pd.DataFrame(pd.Series([10,20], name='float64', dtype='float64'))
print(df.dtypes, '\n')
df2 = pd.read_json(df.to_json(orient='records'), orient='records')
print(df2.dtypes)
print('\nis Json translation reversible ? ', df.equals(df2))

float64    float64
dtype: object 

float64    int64
dtype: object

is Json translation reversible ?  False


In [9]:
sr = pd.Series([math.nan,math.nan], name='nan')
print(sr.dtype, '\n')
sr2 = pd.read_json(sr.to_json(), typ='series')
print(sr2)
print('\nis Json translation reversible ? ', sr.equals(sr2))

float64 

0   NaT
1   NaT
dtype: datetime64[ns]

is Json translation reversible ?  False


- data with 'object' dtype is kept only in certain cases

In [10]:
dfd = pd.DataFrame({'dates': [date(2021, 3, 1), date(2021, 3, 3)]})

print(dfd.to_json(default_handler=date.isoformat), '\n')
print(dfd.to_json(orient='table'), '\n')

dfd2 = pd.read_json(dfd.to_json(orient='table'), orient='table')
print(dfd2)

print('\nis Json translation reversible ? ', dfd.equals(dfd2))

{"dates":{"0":1614556800000,"1":1614729600000}} 

{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"dates","type":"string"}],"primaryKey":["index"],"pandas_version":"1.4.0"},"data":[{"index":0,"dates":"2021-03-01T00:00:00.000"},{"index":1,"dates":"2021-03-03T00:00:00.000"}]} 

                     dates
0  2021-03-01T00:00:00.000
1  2021-03-03T00:00:00.000

is Json translation reversible ?  False


In [11]:
dfd = pd.DataFrame({'tuple': [(2021, 3, 1), (2021, 3, 3)]})
print(dfd, '\n')
print(dfd.to_json(), '\n')
print(dfd.to_json(orient='table'), '\n')
dfd2 = pd.read_json(dfd.to_json(orient='table'), orient='table')
print(dfd2)
print('\nis Json translation reversible ? ', dfd.equals(dfd2))

          tuple
0  (2021, 3, 1)
1  (2021, 3, 3) 

{"tuple":{"0":[2021,3,1],"1":[2021,3,3]}} 

{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"tuple","type":"string"}],"primaryKey":["index"],"pandas_version":"1.4.0"},"data":[{"index":0,"tuple":[2021,3,1]},{"index":1,"tuple":[2021,3,3]}]} 

          tuple
0  [2021, 3, 1]
1  [2021, 3, 3]

is Json translation reversible ?  False


- with categorical dtype, the underlying dtype is not included in json interface

In [12]:
df = pd.DataFrame(pd.Series([10,20], name='float', dtype='float64'), dtype='category')
print(df.to_json(orient='table'))

{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"float","type":"any","constraints":{"enum":[10.0,20.0]},"ordered":false}],"primaryKey":["index"],"pandas_version":"1.4.0"},"data":[{"index":0,"float":10.0},{"index":1,"float":20.0}]}


- `to_csv` and `to_json` methods have different behaviors

In [13]:
df = pd.DataFrame({'dates' : [date(2021, 10, 2), date(2021, 10, 4)],
                   'times' : [time(10, 10, 2), time(11, 10, 4)]})
print('CSV data :\n', df.to_csv())
print('JSON data :\n', df.to_json())
print('JSON data :\n', df.to_json(date_format='iso'))

CSV data :
 ,dates,times
0,2021-10-02,10:10:02
1,2021-10-04,11:10:04

JSON data :
 {"dates":{"0":1633132800000,"1":1633305600000},"times":{"0":"10:10:02","1":"11:10:04"}}
JSON data :
 {"dates":{"0":"2021-10-02T00:00:00.000","1":"2021-10-04T00:00:00.000"},"times":{"0":"10:10:02","1":"11:10:04"}}


### Data compactness
- json-table interface is not compact (in this example the size is triple or quadruple the size of the compact format

In [14]:
tab_data = {'dates':           ['1964-01-01', '1985-02-05', '2022-01-21', '1964-01-01', '1985-02-05', '2022-01-21'], 
            'value':           [10, 10, 20, 20, 30, 30],
            'names':           ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
            'unique':          [True, True, True, True, True, True] }

df = pd.DataFrame(tab_data, dtype='category')
print(df, '\n')

# length with compact interface : 240
print(to_json(df, text=True))
print(len(to_json(df, text=True)), '\n')

# length with actual interface : 946
print(df.to_json(orient='table'))
print(len(df.to_json(orient='table')), '\n')

        dates value   names unique
0  1964-01-01    10    john   True
1  1985-02-05    10    eric   True
2  2022-01-21    20  judith   True
3  1964-01-01    20    mila   True
4  1985-02-05    30  hector   True
5  2022-01-21    30   maria   True 

{":tab": {"index": [0, 1, 2, 3, 4, 5], "dates": [["1964-01-01", "1985-02-05", "2022-01-21"], [1]], "value": [[10, 20, 30], [2]], "names": [["eric", "hector", "john", "judith", "maria", "mila"], [2, 0, 3, 5, 1, 4]], "unique": [[true], [0]]}}
240 

{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"dates","type":"any","constraints":{"enum":["1964-01-01","1985-02-05","2022-01-21"]},"ordered":false},{"name":"value","type":"any","constraints":{"enum":[10,20,30]},"ordered":false},{"name":"names","type":"any","constraints":{"enum":["eric","hector","john","judith","maria","mila"]},"ordered":false},{"name":"unique","type":"boolean","constraints":{"enum":[true]},"ordered":false}],"primaryKey":["index"],"pandas_version":"1.4.0"},"data":[{"in

### Interface is reversible only with json dtype
- interface is reversible with primitive json types (int64, float64, string, bool) and with datetime type
- see previous examples and [Annexe](#5---Annexe:-JSON-interface---option:-"table")

### External types
- the interface does not accept external types
- to integrate external types, it is necessary to first create ExtensionArray and ExtensionDtype objects

## 2 - Series

### Simple example

In [15]:
field_data = {'value': [1, 2, 3]}
sr = read_json({':field': field_data})
# pandas dtype conform to Ntv type
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', to_json(sr))
print('\nis Json translation reversible ? ', sr.equals(read_json(to_json(sr))))

pandas object :
0    1
1    2
2    3
Name: value, dtype: int64

Json representation : 
     {':field': {'value': [1, 2, 3]}}

is Json translation reversible ?  True


### Typed example

In [16]:
field_data = {'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21']}
sr = read_json({':field': field_data})
# pandas dtype conform to Ntv type
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', to_json(sr))
print('\nis Json translation reversible ? ', sr.equals(read_json(to_json(sr))))

pandas object :
0   1964-01-01
1   1985-02-05
2   2022-01-21
Name: dates, dtype: datetime64[ns]

Json representation : 
     {':field': {'dates::datetime': ['1964-01-01T00:00:00.000', '1985-02-05T00:00:00.000', '2022-01-21T00:00:00.000']}}

is Json translation reversible ?  True


### Examples with a non-Pandas type

In [17]:
field_data = {'dates::date': ['1964-01-01', '1985-02-05', '2022-01-21']}
sr = read_json({':field': field_data})
# pandas dtype : object
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', to_json(sr))
print('\nis Json translation reversible ? ', sr.equals(read_json(to_json(sr))))

pandas object :
0    1964-01-01
1    1985-02-05
2    2022-01-21
Name: dates::date, dtype: object

Json representation : 
     {':field': {'dates::date': ['1964-01-01', '1985-02-05', '2022-01-21']}}

is Json translation reversible ?  True


In [18]:
field_data = {'coord::point':    [[1,2], [3,4], [5,6]]}
sr = read_json({':field': field_data})
# pandas dtype : object
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', to_json(sr))
print('\nis Json translation reversible ? ', sr.equals(read_json(to_json(sr))))

pandas object :
0    POINT (1 2)
1    POINT (3 4)
2    POINT (5 6)
Name: coord::point, dtype: object

Json representation : 
     {':field': {'coord::point': [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]}}

is Json translation reversible ?  True


### Categorical examples
- available only with hashable data

In [19]:
field_data = {"integer": [[1, 2], [0, 1, 1, 0]]}
sr = read_json({':field': field_data})
# pandas dtype : object
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', to_json(sr))
print('\nis Json translation reversible ? ', sr.equals(read_json(to_json(sr))))

pandas object :
0    1
1    2
2    2
3    1
Name: integer, dtype: category
Categories (2, Int64): [1, 2]

Json representation : 
     {':field': {'integer': [[1, 2], [0, 1, 1, 0]]}}

is Json translation reversible ?  True


In [20]:
field_data = {'dates': [{'::date': ['1964-01-01', '1985-02-05', '2022-01-21']}, [0, 1, 0, 2]]}
sr = read_json({':field': field_data})
# pandas dtype : object
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', to_json(sr))
print('\nis Json translation reversible ? ', sr.equals(read_json(to_json(sr))))

pandas object :
0    1964-01-01
1    1985-02-05
2    1964-01-01
3    2022-01-21
Name: dates::date, dtype: category
Categories (3, object): [1964-01-01, 1985-02-05, 2022-01-21]

Json representation : 
     {':field': {'dates': [{'::date': ['1964-01-01', '1985-02-05', '2022-01-21']}, [0, 1, 0, 2]]}}

is Json translation reversible ?  True


In [21]:
field_data = {'test_array': [{'::array': [[1,2], [3,4], [5,6]]}, [0, 1, 0, 2]]}
sr = read_json({':field': field_data})
# pandas dtype : object
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', to_json(sr))
print('\nis Json translation reversible ? ', sr.equals(read_json(to_json(sr))))

pandas object :
0    (1, 2)
1    (3, 4)
2    (1, 2)
3    (5, 6)
Name: test_array::array, dtype: category
Categories (3, object): [(1, 2), (3, 4), (5, 6)]

Json representation : 
     {':field': {'test_array': [{'::array': [(1, 2), (3, 4), (5, 6)]}, [0, 1, 0, 2]]}}

is Json translation reversible ?  True


## 3 - DataFrame

### Initial example

In [22]:
df = pd.DataFrame({"A": list("abca"), "B": list("bccd")})

print('pandas dtype :\n' + str(df.dtypes))
print('\npandas object :\n' + str(df))
print('\nJson representation : \n    ', to_json(df))
print('\nis Json translation reversible ? ', df.equals(read_json(to_json(df))))

pandas dtype :
A    object
B    object
dtype: object

pandas object :
   A  B
0  a  b
1  b  c
2  c  c
3  a  d

Json representation : 
     {':tab': {'index': [0, 1, 2, 3], 'A': ['a', 'b', 'c', 'a'], 'B': ['b', 'c', 'c', 'd']}}

is Json translation reversible ?  True


### Complete example
- index data
- Pandas dtype (int32, bool, string)
- NTV type (date, point) -> object dtype
- data unique

In [23]:
tab_data = {'index':           [100, 200, 300, 400, 500, 600],
            'dates::date':     ['1964-01-01', '1985-02-05', '2022-01-21', '1964-01-01', '1985-02-05', '2022-01-21'], 
            'value':           [10, 10, 20, 20, 30, 30],
            'value32::int32':  [12, 12, 22, 22, 32, 32],
            'res':             [10, 20, 30, 10, 20, 30],
            'coord::point':    [[1,2], [3,4], [5,6], [7,8], [3,4], [5,6]],
            'names::string':   ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
            'unique':          True }
df = read_json({':tab': tab_data})
print('pandas dtype :\n' + str(df.dtypes))
print('\npandas object :\n' + str(df))
print('\nJson representation :')
pprint(to_json(df), width=140)
print('\nis Json translation reversible ? ', df.equals(read_json(to_json(df))))

pandas dtype :
dates::date     object
value            int64
value32          int32
res              int64
coord::point    object
names           string
unique            bool
dtype: object

pandas object :
    dates::date  value  value32  res coord::point   names  unique
100  1964-01-01     10       12   10  POINT (1 2)    john    True
200  1985-02-05     10       12   20  POINT (3 4)    eric    True
300  2022-01-21     20       22   30  POINT (5 6)  judith    True
400  1964-01-01     20       22   10  POINT (7 8)    mila    True
500  1985-02-05     30       32   20  POINT (3 4)  hector    True
600  2022-01-21     30       32   30  POINT (5 6)   maria    True

Json representation :
{':tab': {'coord::point': [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0], [3.0, 4.0], [5.0, 6.0]],
          'dates::date': ['1964-01-01', '1985-02-05', '2022-01-21', '1964-01-01', '1985-02-05', '2022-01-21'],
          'index': [100, 200, 300, 400, 500, 600],
          'names::string': ['john', 'eric', 'j

### Json data can be annotated

In [24]:
tab_data = {'index':           [100, 200, 300, 400, 500, 600],
            'dates::date':     ['1964-01-01', '1985-02-05', '2022-01-21', '1964-01-01', '1985-02-05', '2022-01-21'], 
            'value':           [10, 10, 20, 20, {'valid?': 30}, 30],
            'value32::int32':  [12, 12, 22, 22, 32, 32],
            'res':             {'res1': 10, 'res2': 20, 'res3': 30, 'res4': 10, 'res5': 20, 'res6': 30},
            'coord::point':    [[1,2], [3,4], [5,6], [7,8], {'same as 2nd point': [3,4]}, [5,6]],
            'names::string':   ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
            'unique':          True }

df2 = read_json({':tab': tab_data}, annotated=True)
print('is DataFrame identical ? ', df.equals(df2))

is DataFrame identical ?  True


### Categorical data can be included

In [25]:
df = pd.DataFrame({"A": list("abca"), "B": list("bccd")}, dtype="category")

print('pandas dtype :\n' + str(df.dtypes))
print('\npandas object :\n' + str(df))
print('\nJson representation : \n    ', to_json(df))
print('\nis Json translation reversible ? ', df.equals(read_json(to_json(df))))

pandas dtype :
A    category
B    category
dtype: object

pandas object :
   A  B
0  a  b
1  b  c
2  c  c
3  a  d

Json representation : 
     {':tab': {'index': [0, 1, 2, 3], 'A': [['a', 'b', 'c'], [0, 1, 2, 0]], 'B': [['b', 'c', 'd'], [0, 1, 1, 2]]}}

is Json translation reversible ?  True


In [26]:
tab_data = {'index':           [100, 200, 300, 400, 500, 600],
            'dates':           [{'::date': ['1964-01-01', '1985-02-05', '2022-01-21']}, [0, 1, 2, 0, 1, 2]],
            'value':           [[10, 20, {'valid?': 30}], [0, 0, 1, 1, 2, 2]],
            'value32::int32':  [12, 12, 22, 22, 32, 32],
            'res':             {'res1': 10, 'res2': 20, 'res3': 30, 'res4': 10, 'res5': 20, 'res6': 30},
            'coord::point':    [[1,2], [3,4], [5,6], [7,8], {'same as 2nd point': [3,4]}, [5,6]],
            'names::string':   ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
            'unique:boolean':  True }

df = read_json({':tab': tab_data}, annotated=True)
print('pandas dtype :\n' + str(df.dtypes))
print('\npandas object :\n' + str(df))
print('\nJson representation :')
pprint(to_json(df), width=140)
print('\nis Json translation reversible ? ', df.equals(read_json(to_json(df))))

pandas dtype :
dates::date        category
value              category
value32               int32
res                   int64
coord::point         object
names                string
unique::boolean        bool
dtype: object

pandas object :
    dates::date value  value32  res coord::point   names  unique::boolean
100  1964-01-01    10       12   10  POINT (1 2)    john             True
200  1985-02-05    10       12   20  POINT (3 4)    eric             True
300  2022-01-21    20       22   30  POINT (5 6)  judith             True
400  1964-01-01    20       22   10  POINT (7 8)    mila             True
500  1985-02-05    30       32   20  POINT (3 4)  hector             True
600  2022-01-21    30       32   30  POINT (5 6)   maria             True

Json representation :
{':tab': {'coord::point': [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0], [3.0, 4.0], [5.0, 6.0]],
          'dates': [{'::date': ['1964-01-01', '1985-02-05', '2022-01-21']}, [1]],
          'index': [100, 200, 300, 

In [27]:
index   = pd.Series([100, 200, 300, 400, 500, 600])
dates   = pd.Series(name='dates::date',  data=[date(1964, 1, 1), date(1985, 2, 5), date(2022, 1, 21), date(1964, 1, 1),
                                               date(1985, 2, 5), date(2022, 1, 21)], dtype='object').astype('category')
value   = pd.Series(name='value',        data=[10,10,20,20,30,30], dtype='Int64').astype('category') #alias mandatory 
value32 = pd.Series(name='value32',      data=[12, 12, 22, 22, 32, 32], dtype='int32')
coord   = pd.Series(name='coord::point', data=[Point(1,2), Point(3,4), Point(5,6), Point(7,8), Point(3,4), Point(5,6)])
names   = pd.Series(name='names',        data=['john', 'eric', 'judith', 'mila', 'hector', 'maria'], dtype='string')
unique  = pd.Series(name='unique',       data=[True, True, True, True, True, True])

df = pd.DataFrame({ser.name: ser for ser in [index, dates, value, value32, coord, names, unique]}).set_index(None)

print('pandas dtype :\n' + str(df.dtypes))
print('\npandas object :\n' + str(df))
print('\nJson representation :')
pprint(to_json(df), width=140)
print('\nis Json translation reversible ? ', df.equals(read_json(to_json(df))))

pandas dtype :
dates::date     category
value           category
value32            int32
coord::point      object
names             string
unique              bool
dtype: object

pandas object :
    dates::date value  value32 coord::point   names  unique
100  1964-01-01    10       12  POINT (1 2)    john    True
200  1985-02-05    10       12  POINT (3 4)    eric    True
300  2022-01-21    20       22  POINT (5 6)  judith    True
400  1964-01-01    20       22  POINT (7 8)    mila    True
500  1985-02-05    30       32  POINT (3 4)  hector    True
600  2022-01-21    30       32  POINT (5 6)   maria    True

Json representation :
{':tab': {'coord::point': [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0], [3.0, 4.0], [5.0, 6.0]],
          'dates': [{'::date': ['1964-01-01', '1985-02-05', '2022-01-21']}, [1]],
          'index': [100, 200, 300, 400, 500, 600],
          'names::string': ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
          'unique': True,
          'value': [

### Multidimensional data
- JSON-TAB format is applicable for multidimensional data
- JSON multi-dimensional data can be translated into a Pandas Dataframe or a DataArray Xarray

In [28]:
data = {"quantity": ["1 kg", "1 kg", "1 kg", "1 kg", "10 kg", "10 kg", "10 kg", "10 kg"],
        "product": ["banana", "orange", "apple", "peppers", "banana", "orange", "apple", "peppers"], 
        "plants": ["fruit", "fruit", "fruit", "vegetable", "fruit", "fruit", "fruit", "vegetable"], 
        "price": [0.5, 2, 1, 1.5, 5, 20, 10, 15]}

df  = pd.DataFrame(data)
df2 = pd.DataFrame(data, dtype='category').sort_values(by=['quantity', 'product'])
df2

,quantity,product,plants,price
2,1 kg,apple,fruit,1.0
0,1 kg,banana,fruit,0.5
1,1 kg,orange,fruit,2.0
3,1 kg,peppers,vegetable,1.5
6,10 kg,apple,fruit,10.0
4,10 kg,banana,fruit,5.0
5,10 kg,orange,fruit,20.0
7,10 kg,peppers,vegetable,15.0


In [29]:
json_df = Ntv.obj(df).to_obj()[':tab']
print('json_df is the JSON-TAB format with "full" mode\n')
pprint(json_df, width=200)

json_xar = Ntv.obj(df2).to_obj()[':tab']
print('\njson_xa is the JSON-TAB format with "optimize" mode\n')
pprint(json_xar, width=200)

df_from_xar = Ntv.obj({':tab': json_xar}).to_obj(format='obj').sort_index()
print('\nDataFrame from the two JSON-TAB format are identical ? ', df.astype('object').equals(df_from_xar.astype('object')))

print('\nThe "optimize" JSON-TAB format is the image of the DataArray Xarray')
from observation import Sdataset
Sdataset.ntv(json_df).setcanonorder().to_xarray(varname='price')

json_df is the JSON-TAB format with "full" mode

{'index': [0, 1, 2, 3, 4, 5, 6, 7],
 'plants': ['fruit', 'fruit', 'fruit', 'vegetable', 'fruit', 'fruit', 'fruit', 'vegetable'],
 'price': [0.5, 2.0, 1.0, 1.5, 5.0, 20.0, 10.0, 15.0],
 'product': ['banana', 'orange', 'apple', 'peppers', 'banana', 'orange', 'apple', 'peppers'],
 'quantity': ['1 kg', '1 kg', '1 kg', '1 kg', '10 kg', '10 kg', '10 kg', '10 kg']}

json_xa is the JSON-TAB format with "optimize" mode

{'index': [2, 0, 1, 3, 6, 4, 5, 7],
 'plants': [['fruit', 'vegetable'], [0, 0, 0, 1, 0, 0, 0, 1]],
 'price': [[0.5, 1.0, 1.5, 2.0, 5.0, 10.0, 15.0, 20.0], [1, 0, 3, 2, 5, 4, 7, 6]],
 'product': [['apple', 'banana', 'orange', 'peppers'], [1]],
 'quantity': [['1 kg', '10 kg'], [4]]}

DataFrame from the two JSON-TAB format are identical ?  True

The "optimize" JSON-TAB format is the image of the DataArray Xarray


<xarray.DataArray 'price' (quantity: 2, product: 4)>
array([[0.5, 2.0, 1.0, 1.5],
       [5.0, 20.0, 10.0, 15.0]], dtype=object)
Coordinates:
  * quantity  (quantity) object '1 kg' '10 kg'
  * product   (product) object 'banana' 'orange' 'apple' 'peppers'
    plants    (product) object 'fruit' 'fruit' 'fruit' 'vegetable'

## 4 - Appendix : Series tests

In [30]:
# json interface ok
srs = [
       # without ntv_type, without dtype
       pd.Series([{'a': 2, 'e':4}, {'a': 3, 'e':5}, {'a': 4, 'e':6}]),  
       pd.Series([[1,2], [3,4], [5,6]]),  
       pd.Series([[1,2], [3,4], {'a': 3, 'e':5}]),  
       pd.Series([True, False, True]),
       pd.Series(['az', 'er', 'cd']),
       pd.Series(['az', 'az', 'az']),
       pd.Series([1,2,3]),
       pd.Series([1.1,2,3]),
       
       # without ntv_type, with dtype
       pd.Series([10,20,30], dtype='Int64'),
       pd.Series([True, False, True], dtype='boolean'),
       pd.Series([1.1, 2, 3], dtype='float64'), 

       # with ntv_type only in json data (not numbers)
       pd.Series([pd.NaT, pd.NaT, pd.NaT]),
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2)], dtype='datetime64[ns]'),
       pd.Series(pd.to_timedelta(['1D', '2D'])),
       pd.Series(['az', 'er', 'cd'], dtype='string'), 

       # with ntv_type only in json data (numbers)
       pd.Series([1,2,3], dtype='Int32'), 
       pd.Series([1,2,3], dtype='UInt64'),
       pd.Series([1,2,3], dtype='float32'),

       # with ntv_type in Series name and in json data (numbers)
       pd.Series([1,2,3], name='::int64'),
       pd.Series([1,2,3], dtype='Float64', name='::float64'), # force dtype dans la conversion json

       # with ntv_type in Series name and in json data (not numbers)
       pd.Series([[1,2], [3,4], [5,6]], name='::array'),  
       pd.Series([{'a': 2, 'e':4}, {'a': 3, 'e':5}, {'a': 4, 'e':6}], name='::object'),  
       pd.Series([None, None, None], name='::null'), 
       pd.Series(["geo:13.412 ,103.866", "mailto:John.Doe@example.com"], name='::uri', dtype='string'),
       pd.Series(["///path/to/file", "//host.example.com/path/to/file"], name='::file', dtype='string'),

       # with ntv_type converted in object dtype (not in datetime)
       pd.Series([date(2022, 1, 1), date(2022, 1, 2)], name='::date'),
       pd.Series([time(10, 21, 1),  time(8, 1, 2)],    name='::time'),

       # with ntv_type unknown in pandas and with pandas conversion               
       pd.Series([1,2,3], dtype='int64', name='::day'),
       pd.Series([2001,2002,2003], dtype='int64', name='::year'),
       pd.Series([21,10,55], name='::minute'),

       # with ntv_type unknown in pandas and NTV conversion
       pd.Series([Point(1, 0), Point(1, 1), Point(1, 2)], name='::point'),
]
for sr in srs:
    print(as_def_type(sr).equals(read_json(to_json(sr))), 
          read_json(to_json(sr)).name == sr.name, 
          to_json(sr))  

True True {':field': [{'a': 2, 'e': 4}, {'a': 3, 'e': 5}, {'a': 4, 'e': 6}]}
True True {':field': [[1, 2], [3, 4], [5, 6]]}
True True {':field': [[1, 2], [3, 4], {'a': 3, 'e': 5}]}
True True {':field': [True, False, True]}
True True {':field': ['az', 'er', 'cd']}
True True {':field': ['az', 'az', 'az']}
True True {':field': [1, 2, 3]}
True True {':field': [1.1, 2.0, 3.0]}
True True {':field': [10, 20, 30]}
True True {':field': [True, False, True]}
True True {':field': [1.1, 2.0, 3.0]}
True True {':field': {'::datetime': [None, None, None]}}
True True {':field': {'::datetime': ['2022-01-01T00:00:00.000', '2022-01-02T00:00:00.000']}}
True True {':field': {'::duration': ['P1DT0H0M0S', 'P2DT0H0M0S']}}
True True {':field': {'::string': ['az', 'er', 'cd']}}
True True {':field': {'::int32': [1, 2, 3]}}
True True {':field': {'::uint64': [1, 2, 3]}}
True True {':field': {'::float32': [1.0, 2.0, 3.0]}}
True True {':field': {'::int64': [1, 2, 3]}}
True True {':field': {'::float64': [1.0, 2.0, 3.0

In [31]:
# json interface ok
for a in [{'test::int32': [1,2,3]},
          {'test': [1,2,3]},
          [1.0, 2.1, 3.0],
          ['er', 'et', 'ez'],
          [True, False, True],
          {'::boolean': [True, False, True]},
          {'::string': ['er', 'et', 'ez']},
          {'test::float32': [1.0, 2.5, 3.0]},
          {'::int64': [1,2,3]},
          {'::datetime': ["2021-12-31T23:00:00.000","2022-01-01T23:00:00.000"] },
          {'::date': ["2021-12-31", "2022-01-01"] },
          {'::time': ["23:00:00", "23:01:00"] },
          {'::object': [{'a': 3, 'e':5}, {'a': 4, 'e':6}]},
          {'::array': [[1,2], [3,4], [5,6]]},
          True,
          {':boolean': True}
         ]:
    field = {':field': a}
    print(to_json(read_json(field)) == field, field)

True {':field': {'test::int32': [1, 2, 3]}}
True {':field': {'test': [1, 2, 3]}}
True {':field': [1.0, 2.1, 3.0]}
True {':field': ['er', 'et', 'ez']}
True {':field': [True, False, True]}
True {':field': {'::boolean': [True, False, True]}}
True {':field': {'::string': ['er', 'et', 'ez']}}
True {':field': {'test::float32': [1.0, 2.5, 3.0]}}
True {':field': {'::int64': [1, 2, 3]}}
True {':field': {'::datetime': ['2021-12-31T23:00:00.000', '2022-01-01T23:00:00.000']}}
True {':field': {'::date': ['2021-12-31', '2022-01-01']}}
True {':field': {'::time': ['23:00:00', '23:01:00']}}
True {':field': {'::object': [{'a': 3, 'e': 5}, {'a': 4, 'e': 6}]}}
True {':field': {'::array': [[1, 2], [3, 4], [5, 6]]}}
True {':field': True}
True {':field': {':boolean': True}}


In [32]:
# json interface ok (categorical data)
for a in [{'test': [{'::int32': [1, 2, 3]}, [0,1,2,0,1]]},
          {'test': [[1, 2, 3], [0,1,2,0,1]]},
          [[1.0, 2.1, 3.0], [0,1,2,0,1]],
          [['er', 'et', 'ez'], [0,1,2,0,1]],
          [[True, False], [0,1,0,1,0]],
          [{'::string': ['er', 'et', 'ez']}, [0,1,2,0,1]],
          {'test':[{'::float32': [1.0, 2.5, 3.0]}, [0,1,2,0,1]]},
          [{'::int64': [1, 2, 3]}, [0,1,2,0,1]],
          [{'::datetime': ["2021-12-31T23:00:00.000", "2022-01-01T23:00:00.000"] }, [0,1,0,1,0]],
          [{'::date': ["2021-12-31", "2022-01-01"] }, [0,1,0,1,0]],
          [{'::time': ["23:00:00", "23:01:00"] }, [0,1,0,1,0]],
          {'test_date': [{'::datetime': ["2021-12-31T23:00:00.000", "2022-01-01T23:00:00.000"] }, [0,1,0,1,0]]},
          [{'::boolean': [True, False]}, [0,1,0,1,0]],
          [[True], [2]], # periodic Series
          {'quantity': [['1 kg', '10 kg'], [4]]}]:  # periodic Series
    field = {':field': a}
    print(to_json(read_json(field)) == field, field)

True {':field': {'test': [{'::int32': [1, 2, 3]}, [0, 1, 2, 0, 1]]}}
True {':field': {'test': [[1, 2, 3], [0, 1, 2, 0, 1]]}}
True {':field': [[1.0, 2.1, 3.0], [0, 1, 2, 0, 1]]}
True {':field': [['er', 'et', 'ez'], [0, 1, 2, 0, 1]]}
True {':field': [[True, False], [0, 1, 0, 1, 0]]}
True {':field': [{'::string': ['er', 'et', 'ez']}, [0, 1, 2, 0, 1]]}
True {':field': {'test': [{'::float32': [1.0, 2.5, 3.0]}, [0, 1, 2, 0, 1]]}}
True {':field': [{'::int64': [1, 2, 3]}, [0, 1, 2, 0, 1]]}
True {':field': [{'::datetime': ['2021-12-31T23:00:00.000', '2022-01-01T23:00:00.000']}, [0, 1, 0, 1, 0]]}
True {':field': [{'::date': ['2021-12-31', '2022-01-01']}, [0, 1, 0, 1, 0]]}
True {':field': [{'::time': ['23:00:00', '23:01:00']}, [0, 1, 0, 1, 0]]}
True {':field': {'test_date': [{'::datetime': ['2021-12-31T23:00:00.000', '2022-01-01T23:00:00.000']}, [0, 1, 0, 1, 0]]}}
True {':field': [{'::boolean': [True, False]}, [0, 1, 0, 1, 0]]}
True {':field': [[True], [2]]}
True {':field': {'quantity': [['1 kg',

In [33]:
# json interface ko
srs = [# without ntv_type
       pd.Series([math.nan, math.nan]), # bug pandas conversion json : datetime NaT
       
       # without ntv_type, with dtype
       pd.Series([math.nan, math.nan], dtype='float64'), # bug pandas conversion json : datetime NaT
    
       # with ntv_type in Series name and in json data
       pd.Series([1,2,3], dtype='UInt64', name='::uint64'),   # name inutile
       
       # with ntv_type unknown in pandas
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2), datetime(2022, 1, 3)], dtype='datetime64[ns, UTC]'), #à traiter
]
for sr in srs:
    print(as_def_type(sr).equals(read_json(to_json(sr))), 
          read_json(to_json(sr)).name == sr.name, 
          to_json(sr, text=True))  

False True {":field": [null, null]}
False True {":field": [null, null]}
True False {":field": {"::uint64": [1, 2, 3]}}
False True {":field": ["2022-01-01T00:00:00.000", "2022-01-02T00:00:00.000", "2022-01-03T00:00:00.000"]}


In [34]:
# json interface ko (categorical data)
for a in [{'test_array': [{'::array': [[1,2], [3,4], [5,6], [7,8]]}, [0, 1, 0, 2, 3]]}]: # list -> tuple to be hashable
    field = {':field': a}
    print(to_json(read_json(field)) == field, field)

False {':field': {'test_array': [{'::array': [[1, 2], [3, 4], [5, 6], [7, 8]]}, [0, 1, 0, 2, 3]]}}


## 5 - Appendix: JSON interface - option: "table"

### Converting pandas dtype to table-schema type

- a part of pandas dtype (6 in full and 2 partially) are taken into account in the interface 
- `constraints` or `extDtype` members are added into table-schema header

| **pandas**<br><br><br><br>dtype | **pandas**<br><br><br><br>dtype reverse | **pandas**<br><br>Reversible<br>(DataFrame are equal <br>and dtype are equal) | **table-schema**<br><br><br><br>type |
| -------------------- | -------------------- | -------------------- | -------------------- |
| datetime64[ns, <tz>]  | datetime64[ns, <tz>]          | yes | datetime<br>'tz': <tz> |
| datetime64[ns]        | datetime64[ns]                | yes | datetime |
| int64                 | int64                         | yes | integer |
| float64               | float64                       | yes | number |
| bool                  | bool                          | yes | boolean |
| string                | string                        | yes | any<br>'extDtype': 'string' |
| *category*              | category                      | depends on the dtype inside category | any<br>'enum': [xxxx] |
| *object*                | object                        | depends on value type | string |
| *Sparse[<dtype, fill>]* | <dtype>                       | no | type (from dtype)<br>'extDtype': 'Sparse[<dtype, fill>]' |
| *int*                   | int64                         | no | integer |
| *Intxx, UIntxx*         | int64                         | no | integer<br>'extDtype':dtype |
| *float*                 | float64                       | no | number |
| *float32*               | float64                       | no | number |
| *Float64*               | float64                       | no | number<br>'extDtype':dtype |
| *boolean*               | bool                          | no | boolean<br>'extDtype': 'boolean' |
| *period[<freq>]*        | Read-json not available       | no | datetime<br>'freq': 'M' |
| *timedelta64[ns]*       | Read_json not yet implemented | no | duration |
| *interval*              | not available                 | no | string |
    
 
```python
reversibility :
    df_reverse = pd.read_json(df.to_json(orient='table'), orient='table')
```

### Converting table-schema type to pandas dtype

- 24 data types are defined in table-schema, 6 are taken into account in the interface

| **Data**<br><br><br><br>Data type           | **table-schema**<br><br><br><br>format| **table-schema**<br><br><br><br>type | **pandas**<br><br><br>Specification<br>orient=’table’ |  **pandas**<br><br><br>read_json<br>orient=’table’ |
| ------------------------------------------- | ---------------------------------- | -------------------- | -------------------- | -------------------- |
| datetime                                    | default (datetime ISO8601 in UTC)  | datetime             | datetime64[ns] | ok |
| number                                      | default                            | number               | float64 | ok |
| integer                                     | default                            | integer              | int64 | ok |
| boolean                                     | default                            | boolean              | bool | ok |
| string                                      | default                            | string               | object | ok |
| custom type                                 | default                            | any (custom type)    | category / string | ok |
| *email*                                       | email                              | string               |  | Format not supported |
| *uri*                                         | uri                                | string               |  | Format not supported |
| *binary*                                      | Binary (base64 string)             | string               |  | Format not supported |
| *uuid*                                        | uuid                               | string               |  | Format not supported |
| *date, time or datetime with parsable format* | any (parsable ?)                   | date, time, datetime |  | Format partially supported |
| *date, time or datetime with custom format*   | \<PATTERN\>                          | date, time, datetime |  | Format not supported |
| duration                                    | default (lexical duration ISO8601) | duration             | timedelta64[ns] | Read_json not yet implemented |
| *Json data*                                   | default (json)                     | object               |  | Unsupported |
| *Json array*                                  | default (json array)               | array                |  | Unsupported |
| *date*                                        | default (date ISO8601)             | date                 |  | Unsupported |
| *time*                                        | default (time ISO8601)             | time                 |  | Unsupported |
| *year*                                        | default                            | year                 |  | Unsupported |
| *month*                                       | default                            | yearmonth            |  | Unsupported |
| *Point (string)*                              | default (string “lon, lat”)        | geopoint             |  | Unsupported |
| *Point (geojson array)*                       | array (array [lon, lat])           | geopoint             |  | Unsupported |
| *Point (json object)*                         | object (eg {"lon": 90, "lat": 45}) | geopoint             |  | Unsupported |
| *Geometry (geojson)*                          | default (geojson spec)             | geojson              |  | Unsupported |
| *Geometry (topojson)*                         | Topojson (topojson spec)           | geojson              |  | Unsupported |
| *Everything (custom type)*                    | <any> (string)                     | <any>                |  | Only ‘any‘ is supported |

### Example converting pandas dtype to table-schema type

In [35]:
import json

data = [[{'test': [1,2,3]}, {'dtype': 'int32'}],
        [{'test': [1,2,3]}, {'dtype': 'int'}],
        [{'test': [1,2,3]}, {'dtype': 'int64'}],
        [{'test': [1,2,3]}, {'dtype': 'Int64'}],
        [{'test': [1,2,3]}, {'dtype': 'UInt64'}],
        [{'test': [1,2,3]}, {'dtype': 'float'}],
        [{'test': [1,2,3]}, {'dtype': 'Float32'}],
        [{'test': [1,2,3]}, {'dtype': 'float32'}],
        [{'test': [1,2,3]}, {'dtype': 'float64'}],
        [{'test': [1,2,3]}, {'dtype': 'Sparse[float64]'}],
        # [{'test': [pd.Interval(1,2), pd.Interval(2,3), pd.Interval(3,4)]}], # read_json ko
        [{'test': ['2020-01-01']}, {'dtype': 'datetime64[ns]'}],
        [{'test': ['2020-01-01']}, {'dtype': 'datetime64[ns, UTC]'}],
        [{'test': ['2020-01-01']}, {'dtype': 'category'}],
        [{'test': [datetime(2020, 1, 1)]}, {'dtype': 'category'}],         
        [{'test': [True, False]}, {'dtype': 'boolean'}],
        [{'test': [True, False]}, {'dtype': 'bool'}],
        # [{'test': ['1 days', '2 days']}, {'dtype': 'timedelta64[ns]'}], # read_json not yet implemented
        # [{'test': ['2020-01-01', '2020-02-01', '2020-03-01']}, {'dtype': 'period[M]'}], # read_json not available
         [{'test': [True, 1, 'er', datetime(2020, 1, 1)]}, {'dtype': 'object'}],
       ]
print('reverse (True, False), json table-schema object, reverse dtype:\n')
for df_data in data:
    df = pd.DataFrame(df_data[0], **df_data[1]) if len(df_data) == 2 else pd.DataFrame(df_data[0])
    js = df.to_json(orient='table')
    df2 = pd.read_json(js, orient='table')
    print(df.equals(df2), json.loads(js)['schema']['fields'][1], df2.dtypes[0])

reverse (True, False), json table-schema object, reverse dtype:

False {'name': 'test', 'type': 'integer'} int64
False {'name': 'test', 'type': 'integer'} int64
True {'name': 'test', 'type': 'integer'} int64
False {'name': 'test', 'type': 'integer', 'extDtype': 'Int64'} int64
False {'name': 'test', 'type': 'integer', 'extDtype': 'UInt64'} int64
True {'name': 'test', 'type': 'number'} float64
False {'name': 'test', 'type': 'number', 'extDtype': 'Float32'} float64
False {'name': 'test', 'type': 'number'} float64
True {'name': 'test', 'type': 'number'} float64
False {'name': 'test', 'type': 'number', 'extDtype': 'Sparse[float64, nan]'} float64
True {'name': 'test', 'type': 'datetime'} datetime64[ns]
True {'name': 'test', 'type': 'datetime', 'tz': 'UTC'} datetime64[ns, UTC]
True {'name': 'test', 'type': 'any', 'constraints': {'enum': ['2020-01-01']}, 'ordered': False} category
False {'name': 'test', 'type': 'any', 'constraints': {'enum': ['2020-01-01T00:00:00.000']}, 'ordered': False} cate